In [1]:
#import libraries
import carla
import cv2
import numpy as np
import math

In [2]:
#define clinet
client = carla.Client('localhost',2000)
world = client.get_world()
bp_library = world.get_blueprint_library()
vehicle_bp_library = world.get_blueprint_library().filter('vehicle')
transform = world.get_map().get_spawn_points()

In [3]:
#spawn main vehicle
start_point = transform[0]
vehicle_bp = vehicle_bp_library.filter('vehicle.bmw.grandtourer')
main_vehicle = world.spawn_actor(vehicle_bp[0],start_point)

In [4]:
#spawn test vehicle
test_vehicle_bp = vehicle_bp_library.filter('vehicle.mini.cooper_s')
test_vehicle_points = carla.Transform(carla.Location(x=-52.280296, y=46.886234, z=0.067663), carla.Rotation(pitch=-0.004399, yaw=-90.062912, roll=0.0000))
test_vehicle = world.spawn_actor(test_vehicle_bp[0],test_vehicle_points)

In [5]:
#spawn the pedestrian
pedestrian_bp_library = world.get_blueprint_library().filter('*walker*')
pedestrian_bp = pedestrian_bp_library.filter('walker.pedestrian.0024')
pedestrian_points = carla.Transform(carla.Location(x=58.312057, y=24.831600, z=0.067690), carla.Rotation(pitch=-0.001373, yaw=180.000000, roll=0.000000))
pedestrian = world.spawn_actor(pedestrian_bp[0],pedestrian_points)


In [6]:
#define RGB Camera
camera_bp = bp_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '720')
camera_bp.set_attribute('image_size_y', '460')
camera_transform = carla.Transform(carla.Location(x=-5, z=3))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=main_vehicle)
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
camera.listen(lambda image: camera_callback(image,camera_data))

In [7]:
#control speed 
PREFERRED_SPEED = 30 
SPEED_THRESHOLD = 2
def maintain_speed(s):
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8 
    else:
        return 0.4 

In [8]:
#control main vehicle
import sys
sys.path.append('C:\carla\WindowsNoEditor\PythonAPI\carla') #depends on your path
from agents.navigation.global_route_planner import GlobalRoutePlanner

brake_pos = carla.Transform(carla.Location(x=95.991333, y=24.470984, z=0.068305), carla.Rotation(pitch=0.000075, yaw=0.159189, roll=-0.000488))
point_a = start_point.location

sampling_resolution = .8
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

route = grp.trace_route(point_a, brake_pos.location)


for waypoint in route:
    main_vehicle.set_transform(waypoint[0].transform)
    cv2.imshow('main vehicle',camera_data['image'])
    cv2.waitKey(40)
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        break
    #main_vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0,hand_brake=True))
    v = main_vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)
    main_vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                               steer=0.0))

    
cv2.destroyAllWindows()
camera.stop()

In [9]:
#clear all actors
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for pedestrian in world.get_actors().filter('*walker*'):
    pedestrian.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()